In [18]:
from gc_utils import *

alg = Algebra(4)
ga2 = Algebra(2)
locals().update(alg.blades)

In [33]:
# 2d general linear transform
def create_linear_trans(alg: Algebra):
    matrix = np.random.rand(alg.d, alg.d)
    return matrix2trans(matrix, alg)

def create_ortho_trans(alg: Algebra):
    from scipy.stats import ortho_group
    matrix = ortho_group.rvs(dim=alg.d)
    return matrix2trans(matrix, alg)

f = create_linear_trans(ga2)
x = create_r_blade(1, ga2)
f(x)

0.0289 𝐞₁ + 0.461 𝐞₂

In [34]:
# 2d polar decomposition
# f = SUU
df = derivative(f, x, ga2, grade=1)
UU = df / norm(df)
S = lambda x: UU * f(x)
S(x)*UU, f(x)

(0.0289 𝐞₁ + 0.461 𝐞₂, 0.0289 𝐞₁ + 0.461 𝐞₂)

In [35]:
# UU is rotor, S is symmetric
UU, curl(S, x, ga2, grade=1)

(0.817 + 0.576 𝐞₁₂, -2.91e-11 𝐞₁₂)

In [36]:
#6.3 remove the condition of 2 dimension
a, b, x = create_r_vectors(3, alg)
g = lambda x: x + (x|a)*b
abframe = [a,b]
r_frame = reciprocal(abframe)
dg = 2 + a*b
derivative(g, x, alg, grade=1, frame=abframe, r_frame=r_frame), dg

(3.05 + -0.0498 𝐞₁₂ + -0.544 𝐞₁₃ + 0.559 𝐞₁₄ + -0.539 𝐞₂₃ + 0.617 𝐞₂₄ + 0.687 𝐞₃₄ + -2.64e-16 𝐞₁₂₃₄,
 3.05 + -0.0498 𝐞₁₂ + -0.544 𝐞₁₃ + 0.559 𝐞₁₄ + -0.539 𝐞₂₃ + 0.617 𝐞₂₄ + 0.687 𝐞₃₄)

In [37]:
#6.3
dg_norm = norm(dg)
UU = dg / dg_norm
U = simple_rotor_sqrt(UU)
S = lambda x: U.sw(g(x))
curl(S, x, alg, grade=1), U.reverse().sw(S(x)), g(x)

(5.82e-11 𝐞₁₂ + 8.73e-11 𝐞₁₃ + -1.16e-10 𝐞₁₄ + 2.33e-10 𝐞₂₃ + -5.82e-11 𝐞₂₄ + 2.91e-10 𝐞₃₄,
 0.5 𝐞₁ + 0.744 𝐞₂ + 0.577 𝐞₃ + 0.717 𝐞₄,
 0.5 𝐞₁ + 0.744 𝐞₂ + 0.577 𝐞₃ + 0.717 𝐞₄)

In [38]:
#6.6
frame = alg.frame
i, j = 1, 3
aij = (1/np.sqrt(2))*(frame[i] + frame[j])
[aij.sw(x) for x in frame]

[-1.0 𝐞₁, 1.0 𝐞₄, -1.0 𝐞₃, 1.0 𝐞₂]

In [39]:
#6.7
alpha = 3
Sij = lambda x: x + (alpha * (x|frame[i]) * frame[j])
[Sij(x) for x in frame]

[1 𝐞₁, 1 𝐞₂ + 3 𝐞₄, 1 𝐞₃, 1 𝐞₄]

In [40]:
def vectors_partial(F, vectors, directions, h=1e-6):
    r = len(vectors)
    drF = 0
    for offset in product([1,-1], repeat=r):
        offset = np.array(offset)
        coef = np.prod(offset)
        offset = offset * h
        drF += 1/(2*h)**r * coef * F([v+d*o for v, d, o in zip(vectors, directions, offset)])
    return drF

In [47]:
#7.2
g = create_linear_trans(alg)
g_ = lambda x, y: x | g(y)
derivative(lambda x: g_(x, b), x, alg, grade=1), g(b)

(1.11 𝐞₁ + 0.887 𝐞₂ + 1.21 𝐞₃ + 1.73 𝐞₄,
 1.11 𝐞₁ + 0.887 𝐞₂ + 1.21 𝐞₃ + 1.73 𝐞₄)

In [48]:
# nonsingular
outermorphism(g, alg.pseudoscalar((1,)), alg)

-0.0575 𝐞₁₂₃₄

In [51]:
#7.5 orthogonal transformation is its isometry
f = create_ortho_trans(alg)
x | f(b), f(x) | f(f(b))

(-0.185, -0.185)

In [93]:
# skew bilinear form
gm = skew_part(g, alg)
G = simplicial_derivative(lambda vectors: vectors[0] | gm(vectors[1]), np.zeros(2), alg)
x | gm(b), x | (b | G)

(-0.0331, -0.0331)

In [94]:
#7.6b
adjoint_outermorphism(f, G, alg), G

(0.12 𝐞₁₂ + -0.2 𝐞₁₃ + 0.188 𝐞₁₄ + -0.111 𝐞₂₃ + -0.0819 𝐞₂₄ + -0.0731 𝐞₃₄,
 0.0412 𝐞₁₂ + 0.134 𝐞₁₃ + 0.0893 𝐞₁₄ + -0.215 𝐞₂₃ + 0.202 𝐞₂₄ + 0.0012 𝐞₃₄)

In [103]:
#7.7
blades = bivector_split(G, alg)

gm(gm(x)), sum(P(x, blade) for blade in blades)

(-0.00137 𝐞₁ + -0.0276 𝐞₂ + -0.0316 𝐞₃ + 0.00855 𝐞₄,
 -0.00137 𝐞₁ + -0.0276 𝐞₂ + -0.0316 𝐞₃ + 0.00855 𝐞₄)

In [112]:
#7.8
gm(sum(-x | blade/normsq(blade) for blade in blades)), x

(0.028 𝐞₁ + 0.274 𝐞₂ + 0.543 𝐞₃ + 0.157 𝐞₄,
 0.028 𝐞₁ + 0.274 𝐞₂ + 0.543 𝐞₃ + 0.157 𝐞₄)

In [110]:
gm(x)

-0.0979 𝐞₁ + 0.0861 𝐞₂ + -0.0552 𝐞₃ + 0.0584 𝐞₄

In [68]:
#7.12
gprod(blades)

1.04e-17 + -1.08e-16 𝐞₁₂ + -3.47e-17 𝐞₁₃ + 3.47e-18 𝐞₁₄ + 2.78e-17 𝐞₂₄ + 1.39e-17 𝐞₃₄ + -1.0 𝐞₁₂₃₄

In [ ]:
#7.17
theta = np.random.rand(2)
rotors = [blade_exp()]